<a href="https://colab.research.google.com/github/wlvh/RBT/blob/main/RL/llama3_1_pipline_for_RLdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers bitsandbytes accelerate
!pip install -U bitsandbytes

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import torch
import json

In [2]:
from huggingface_hub import login
from google.colab import userdata
login(token=userdata.get('token'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
# 将 pipeline 初始化移到函数外部，避免重复加载模型，提升性能。
# generator = pipeline('text-generation', model="meta-llama/Meta-Llama-3.1-8B-Instruct",max_length=2048,)
# 检查是否有可用的 GPU
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# 初始化 pipeline
generator = pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={
        "torch_dtype": torch.bfloat16,
        "quantization_config": {"load_in_8bit": True}
    },
    device_map="auto",
    max_length=1024*4,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
def generate_market_summary(market_data, generator):
      market_analysis = market_data['market_analysis']

      recent_100 = market_analysis['recent_100_minutes']
      past_7_days = market_analysis['past_7_days']
      market_state = market_analysis['market_state']

      overall_return = recent_100['overall_return']
      positive_returns_ratio = recent_100['positive_returns_ratio']
      price_amplitude = recent_100['price_amplitude']

      return_trend = market_state['return_trend']
      if '横盘' in return_trend:
          trend_direction = '横盘'
      elif '上升' in return_trend:
          trend_direction = '上升'
      elif '下降' in return_trend:
          trend_direction = '下降'
      else:
          trend_direction = '未知'

      majority_or_minority = '均衡' if '均衡' in market_state['trade_majority'] else '其他'

      volume_state = market_state['volume_state']
      volume_change_direction = '增加' if '增加' in volume_state else '减少'

      market_sentiment = market_state['market_sentiment']
      price_comparison = recent_100['price_comparison']

      price_trend = past_7_days['price_trend']
      volume_trend = past_7_days['volume_trend']

      price_volatility = past_7_days['price_volatility']
      volume_volatility = past_7_days['volume_volatility']

      # 构建提示
      prompt = f"""你是一位经验丰富的金融分析师，专门负责为交易员提供市场洞察。请基于以下市场数据，生成一份简洁而全面的市场状态总结报告。

市场数据：
{json.dumps(market_analysis, ensure_ascii=False, indent=2)}

关键观察：
1. 过去100分钟收益率为{overall_return}，正收益比率为{positive_returns_ratio}，价格振幅为{price_amplitude}，市场呈现{trend_direction}趋势。
2. 市场呈现{trend_direction}趋势，{'交易' if trend_direction != '横盘' else '波动'}由{majority_or_minority}决定。
3. 交易量：{volume_change_direction}，交易金额加权价格相比当前价格{price_comparison}。
4. 市场情绪：整体市场情绪偏{market_sentiment}。
5. 过去7天表现：价格呈{price_trend}趋势，成交量呈{volume_trend}趋势。
6. 波动性：成交价格波动率{price_volatility}，成交量波动率{volume_volatility}。

请提供一份市场状态总结报告，包括以下几个方面：
1. 短期趋势分析：基于近期100分钟的数据，分析短期市场走向。
2. 中期趋势预测：结合过去7天的数据，对中期市场趋势进行预测。
3. 市场情绪评估：评估当前的市场情绪，并解释可能的原因。
4. 总体市场态势：基于你分析的1-3，简要描述当前市场的整体状态和主要特征。

请确保你的分析既有洞察力又易于理解，适合快速决策使用。报告应当简明扼要，不需要输出除了1-4外的其他建议。
"""
      # 生成文本
      print(prompt)
      result = generator(prompt, max_length=2048, num_return_sequences=1)

      return result[0]['generated_text']


In [5]:
market_analysis = [
    {
        "trade_info": {
            "trade_start_time": "2023-07-09 00:00:00",
            "trade_end_time": "2023-07-10 16:00:00",
            "direction": "buy",
            "open_cost_price": 30305.14,
            "close_price": 30320.51,
            "profit": 15.3650000000016
        },
        "market_analysis": {
            "recent_5_minutes": {
                "recent_5min_close": [
                    30250.67,
                    30286.88,
                    30289.79,
                    30287.43,
                    30284.63
                ],
                "recent_5min_volume": [
                    32.16553,
                    54.50681,
                    14.55152,
                    5.5892,
                    7.33802
                ]
            },
            "recent_100_minutes": {
                "overall_return": 0.0037,
                "positive_returns_ratio": 0.4949,
                "price_amplitude": 0.0041,
                "volume_change_rate": 4.4485,
                "price_comparison": "低"
            },
            "past_7_days": {
                "price_trend": "维持不动",
                "volume_trend": "下跌",
                "price_volatility": "低",
                "volume_volatility": "高"
            },
            "market_state": {
                "return_trend": "收益率为0.0037，正收益比率为0.4949，振幅为0.0041，市场呈现横盘趋势",
                "trade_majority": "市场呈现横盘趋势，波动由均衡决定",
                "volume_state": "交易量增加",
                "market_sentiment": "乐观"
            }
        },
        "strategy_signals": {
            "RL_sharpratio": {
                "2023-07-09 00:00:00_35_weighted_win_ratio_WRSIStrategy_apply_3": {
                    "buy_avg_volume": 4.8,
                    "ma_long": 30209.0,
                    "ma_short": 30556.7,
                    "rsi": 36.6,
                    "sell_avg_volume": 4.8
                },
                "2023-07-09 00:00:00_70_weighted_SharpeDIY_RSIV_StrategyFourhour_apply_3": {
                    "buy_avg_volume": 4.8,
                    "ma_long": 30208.2,
                    "ma_short": 30177.2,
                    "rsi": 41.8,
                    "sell_avg_volume": 4.8
                }
            }
        },
        "decision": {
            "trade_decision": "buy",
            "selected_strategy": "2023-07-09 00:00:00_35_weighted_win_ratio_WRSIStrategy_apply_3"
        },
        "llm_tasks": {
            "market_summary": "请LLM根据以上信息生成市场状态的总结性评价",
            "strategy_analysis": "请模型综合所有交易策略的数据进行分析评估当前适用的策略和信号",
            "trade_analysis": "交易决策的理由由LLM生成"
        }
    },
    {
        "trade_info": {
            "trade_start_time": "2023-07-10 16:00:00",
            "trade_end_time": "2023-07-14 21:14:00",
            "direction": "sell",
            "open_cost_price": 30305.35,
            "close_price": 30261.53,
            "profit": 43.819745000000694
        },
        "market_analysis": {
            "recent_5_minutes": {
                "recent_5min_close": [
                    30290.15,
                    30297.99,
                    30304.02,
                    30318.77,
                    30312.81
                ],
                "recent_5min_volume": [
                    8.41058,
                    6.91152,
                    8.29558,
                    22.92794,
                    47.38863
                ]
            },
            "recent_100_minutes": {
                "overall_return": 0.0005,
                "positive_returns_ratio": 0.5354,
                "price_amplitude": 0.0054,
                "volume_change_rate": 0.0394,
                "price_comparison": "低"
            },
            "past_7_days": {
                "price_trend": "维持不动",
                "volume_trend": "下跌",
                "price_volatility": "低",
                "volume_volatility": "高"
            },
            "market_state": {
                "return_trend": "收益率为0.0005，正收益比率为0.5354，振幅为0.0054，市场呈现横盘趋势",
                "trade_majority": "市场呈现横盘趋势，波动由均衡决定",
                "volume_state": "交易量增加",
                "market_sentiment": "乐观"
            }
        },
        "strategy_signals": {
            "RL_sharpratio": {
                "2023-07-09 00:00:00_35_weighted_win_ratio_WRSIStrategy_apply_3": {
                    "buy_avg_volume": 35.9,
                    "ma_long": 30156.0,
                    "ma_short": 30414.5,
                    "rsi": 64.0,
                    "sell_avg_volume": 35.9
                },
                "2023-07-09 00:00:00_70_weighted_SharpeDIY_RSIV_StrategyFourhour_apply_3": {
                    "buy_avg_volume": 35.9,
                    "ma_long": 30155.8,
                    "ma_short": 30243.1,
                    "rsi": 45.9,
                    "sell_avg_volume": 35.9
                }
            }
        },
        "decision": {
            "trade_decision": "sell",
            "selected_strategy": "2023-07-09 00:00:00_35_weighted_win_ratio_WRSIStrategy_apply_3"
        },
        "llm_tasks": {
            "market_summary": "请LLM根据以上信息生成市场状态的总结性评价",
            "strategy_analysis": "请模型综合所有交易策略的数据进行分析评估当前适用的策略和信号",
            "trade_analysis": "交易决策的理由由LLM生成"
        }
    }
]

In [6]:
summary = generate_market_summary(market_analysis[0], generator)
print(summary)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


你是一位经验丰富的金融分析师，专门负责为交易员提供市场洞察。请基于以下市场数据，生成一份简洁而全面的市场状态总结报告。
  
市场数据：
{
  "recent_5_minutes": {
    "recent_5min_close": [
      30250.67,
      30286.88,
      30289.79,
      30287.43,
      30284.63
    ],
    "recent_5min_volume": [
      32.16553,
      54.50681,
      14.55152,
      5.5892,
      7.33802
    ]
  },
  "recent_100_minutes": {
    "overall_return": 0.0037,
    "positive_returns_ratio": 0.4949,
    "price_amplitude": 0.0041,
    "volume_change_rate": 4.4485,
    "price_comparison": "低"
  },
  "past_7_days": {
    "price_trend": "维持不动",
    "volume_trend": "下跌",
    "price_volatility": "低",
    "volume_volatility": "高"
  },
  "market_state": {
    "return_trend": "收益率为0.0037，正收益比率为0.4949，振幅为0.0041，市场呈现横盘趋势",
    "trade_majority": "市场呈现横盘趋势，波动由均衡决定",
    "volume_state": "交易量增加",
    "market_sentiment": "乐观"
  }
}
  
关键观察：
1. 过去100分钟收益率为0.0037，正收益比率为0.4949，价格振幅为0.0041，市场呈现横盘趋势。
2. 市场呈现横盘趋势，波动由均衡决定。
3. 交易量：增加，交易金额加权价格相比当前价格低。
4. 市场情绪：整体市场情绪偏乐观。
5. 过去7天表现：价格呈维持不动趋势，成交量呈下跌

/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


你是一位经验丰富的金融分析师，专门负责为交易员提供市场洞察。请基于以下市场数据，生成一份简洁而全面的市场状态总结报告。
  
市场数据：
{
  "recent_5_minutes": {
    "recent_5min_close": [
      30250.67,
      30286.88,
      30289.79,
      30287.43,
      30284.63
    ],
    "recent_5min_volume": [
      32.16553,
      54.50681,
      14.55152,
      5.5892,
      7.33802
    ]
  },
  "recent_100_minutes": {
    "overall_return": 0.0037,
    "positive_returns_ratio": 0.4949,
    "price_amplitude": 0.0041,
    "volume_change_rate": 4.4485,
    "price_comparison": "低"
  },
  "past_7_days": {
    "price_trend": "维持不动",
    "volume_trend": "下跌",
    "price_volatility": "低",
    "volume_volatility": "高"
  },
  "market_state": {
    "return_trend": "收益率为0.0037，正收益比率为0.4949，振幅为0.0041，市场呈现横盘趋势",
    "trade_majority": "市场呈现横盘趋势，波动由均衡决定",
    "volume_state": "交易量增加",
    "market_sentiment": "乐观"
  }
}
  
关键观察：
1. 过去100分钟收益率为0.0037，正收益比率为0.4949，价格振幅为0.0041，市场呈现横盘趋势。
2. 市场呈现横盘趋势，波动由均衡决定。
3. 交易量：增加，交易金额加权价格相比当前价格低。
4. 市场情绪：整体市场情绪偏乐观。
5. 过去7天表现：价格呈维持不动趋势，成交量呈下跌

In [8]:
summary

'你是一位经验丰富的金融分析师，专门负责为交易员提供市场洞察。请基于以下市场数据，生成一份简洁而全面的市场状态总结报告。\n  \n市场数据：\n{\n  "recent_5_minutes": {\n    "recent_5min_close": [\n      30250.67,\n      30286.88,\n      30289.79,\n      30287.43,\n      30284.63\n    ],\n    "recent_5min_volume": [\n      32.16553,\n      54.50681,\n      14.55152,\n      5.5892,\n      7.33802\n    ]\n  },\n  "recent_100_minutes": {\n    "overall_return": 0.0037,\n    "positive_returns_ratio": 0.4949,\n    "price_amplitude": 0.0041,\n    "volume_change_rate": 4.4485,\n    "price_comparison": "低"\n  },\n  "past_7_days": {\n    "price_trend": "维持不动",\n    "volume_trend": "下跌",\n    "price_volatility": "低",\n    "volume_volatility": "高"\n  },\n  "market_state": {\n    "return_trend": "收益率为0.0037，正收益比率为0.4949，振幅为0.0041，市场呈现横盘趋势",\n    "trade_majority": "市场呈现横盘趋势，波动由均衡决定",\n    "volume_state": "交易量增加",\n    "market_sentiment": "乐观"\n  }\n}\n  \n关键观察：\n1. 过去100分钟收益率为0.0037，正收益比率为0.4949，价格振幅为0.0041，市场呈现横盘趋势。\n2. 市场呈现横盘趋势，波动由均衡决定。\n3. 交易量：增加，交易金额加权价格相比当前价格低。\

In [ ]:
'''
你是一位经验丰富的金融分析师，专门负责为交易员提供市场洞察。请基于以下市场数据，生成一份简洁而全面的市场状态总结报告。

市场数据：
{在这里插入结构化的 market_analysis 数据}

关键观察：
1. 近期100分钟表现：平均收益率为{正/负}，中位数收益率为{正/负}。
2. 市场趋势：市场呈现{上涨/下跌/横盘}趋势，这一趋势由{多数/少数}交易决定。
3. 交易量：交易量{增加/减少}，交易金额加权价格相比当前价格{高/低}。
4. 市场情绪：整体市场情绪偏{乐观/悲观}。
5. 过去7天表现：价格呈{上涨/下跌/维持不动}趋势，成交量呈{上涨/下跌}趋势。
6. 波动性：成交价格波动率{高/中/低}，成交量波动率{高/中/低}。

请提供一份市场状态总结报告，包括以下几个方面：
1. 总体市场态势：简要描述当前市场的整体状态和主要特征。
2. 短期趋势分析：基于近期100分钟的数据，分析短期市场走向。
3. 中期趋势预测：结合过去7天的数据，对中期市场趋势进行预测。
4. 市场情绪评估：评估当前的市场情绪，并解释可能的原因。
5. 潜在风险：指出当前市场状态下的主要风险因素。
6. 潜在机会：分析可能存在的投资或交易机会。
7. 建议：为交易员提供2-3条基于当前市场状况的具体建议。

请确保你的分析既有洞察力又易于理解，适合快速决策使用。报告应当简明扼要，总字数控制在300-400字左右。
'''

提供结构化的 "market_analysis" 信息，同时附带一个简洁的文字描述。这种方法有以下优点：

保持数据的结构化特性，便于 LLM 进行精确分析。
提供人类可读的描述，有助于 LLM 理解上下文和生成更自然的语言总结。
灵活性强，可以根据需要调整文字描述的详细程度。

以下是建议的输入格式：

请根据以下市场分析信息生成一个全面的市场状态总结性评价：

结构化数据：
{
    "recent_100_minutes": {
        "mean_return": 0.0,
        "median_return": 0.0,
        "volume_change_rate": 0.748,
        "price_comparison": "低"
    },
    "past_7_days": {
        "price_trend": "维持不动",
        "volume_trend": "下跌",
        "price_volatility": "低",
        "volume_volatility": "高"
    },
    "market_state": {
        "return_trend": "平均收益率为正，中位数收益率为正，市场呈现上涨趋势",
        "trade_majority": "市场上涨由多数交易决定",
        "volume_state": "交易量增加",
        "market_sentiment": "乐观"
    }
}

简要描述：
- 近期100分钟内，平均收益率为正，中位数收益率为正，市场呈现上涨趋势。
- 市场上涨由多数交易决定。
- 交易量增加，交易金额加权价格相比当前价格低，市场情绪偏乐观。
- 过去7天价格呈维持不动趋势，成交量呈下跌趋势，成交价格波动率低，成交量波动率高。

请基于以上信息，提供一个全面的市场状态评估，包括短期和中期趋势、市场情绪、潜在风险和机会等方面。